In [3]:
import os
import sys
import json
import subprocess
import pandas as pd
import numpy as np
import cv2

sys.path.append('../va_pipeline/')
from calculate_accuracy import *
from sensor import exec_file



In [6]:
max_frames = 250
frame_no = 1
cap = cv2.VideoCapture('./large_fast.mp4')

while frame_no < max_frames:
    ret, frame = cap.read()
    if not ret:
        break
    frame_no += 1

cap.release()


In [7]:
print(frame_no)

250
